In [5]:
import cmath
import numpy as np
from numpy.linalg import inv
from numpy import real, trace, dot, eye, conj, transpose, diag
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, VBox, HBox

# Constants (all MKS, except energy which is in eV)
hbar = 1.06e-34                         # Planck constant over 2 pi (J*s)
q = 1.6e-19                             # Elementary charge (C)
m = 1.0 * 9.1e-31                      # Effective mass (kg)
IE = (q * q) / (2 * np.pi * hbar)       # Coulomb interaction energy factor
Ef = 0.1                                # Fermi energy (eV)
kT = 0.025                              # Thermal energy (eV)

# Lattice constant and hopping parameter
a = 3e-10                               # Lattice constant (m)
t0 = (hbar**2) / (2 * m * (a**2) * q)   # Hopping parameter (eV)

def transmission_IV(V_opt, NC, Height, UB_choice):
    NS = ND = (50 - NC) // 2            # Ensure NS = ND and NC + NS + ND = 50
    Np = NS + NC + ND                   # Total number of sites
    print(NS, NC, ND)
    # Tunneling barrier potential
    UB1 = np.vstack((np.zeros((NS, 1)), Height * np.ones((NC, 1)), np.zeros((ND, 1)))).flatten()
    if NC > 8:
        UB2 = np.vstack((np.zeros((NS, 1)), Height * np.ones((4, 1)), np.zeros((NC-8, 1)), Height * np.ones((4, 1)), np.zeros((ND, 1)))).flatten()
    else:
        UB2 = np.vstack((np.zeros((NS, 1)), Height * np.ones((NC, 1)), np.zeros((ND, 1)))).flatten()

    UB = UB1 if UB_choice == 'UB1' else UB2

    # Hamiltonian matrix setup
    T = (2 * t0 * np.diag(np.ones((Np)))) - (t0 * np.diag(np.ones((Np-1)), 1)) - (t0 * np.diag(np.ones((Np-1)), -1))
    T = T + np.diagflat(UB[:Np])

    # Bias voltage setup
    V = V_opt
    mu1 = Ef + (V / 2)
    mu2 = Ef - (V / 2)
    U1 = V * np.hstack((0.5 * np.ones(NS), np.linspace(0.5, -0.5, NC), -0.5 * np.ones(ND)))
    U1 = U1.T  # Applied potential profile

    # Energy grid for Green’s function method
    NE = 501
    E = np.linspace(-0.2, 0.8, NE)
    zplus = 1j * 1e-12

    # Initializing arrays
    TM = np.zeros(NE)

    for k in range(NE):
        sig1 = np.zeros((Np, Np), dtype=complex)
        sig2 = np.zeros((Np, Np), dtype=complex)
        sig3 = np.zeros((Np, Np), dtype=complex)
        
        ck = 1 - ((E[k] + zplus - U1[0] - UB[0]) / (2 * t0))
        ka = cmath.acos(ck)
        sig1[0, 0] = -t0 * np.exp(1j * ka)
        gam1 = 1j * (sig1 - conj(sig1.T))

        ck = 1 - ((E[k] + zplus - U1[Np-1] - UB[Np-1]) / (2 * t0))
        ka = cmath.acos(ck)
        sig2[Np-1, Np-1] = -t0 * np.exp(1j * ka)
        gam2 = 1j * (sig2 - conj(sig2.T))

        if UB_choice == 'UB1':
            G = inv(((E[k] + zplus) * eye(Np)) - T - diag(U1) - sig1 - sig2)
        else:
            G = inv(((E[k] + zplus) * eye(Np)) - T - diag(U1) - sig1 - sig2 - sig3)
        TM[k] = real(trace(dot(dot(dot(gam1, G), gam2), conj(G.T))))
        
    XX = a * 1e9 * np.array(list(range(1, Np + 1)))
    print(XX, len(XX), XX[0], XX[NS-1],XX[NS],XX[NS+NC-1],XX[NS+NC],XX[NS+NC]-XX[NS-1])
    XS = XX[0:NS-4]
    XD = XX[NS+NC+5-1:Np]

    return XX, XS, XD, U1, UB1, UB2, TM, E, mu1, mu2

def calculate_IV(V_opt, NC, Height, UB_choice):
    NS = ND = (50 - NC) // 2    # Ensure NS = ND and NC + NS + ND = 50
    Np = NS + NC + ND           # Total number of sites

    # Tunneling barrier potential
    UB1 = np.vstack((np.zeros((NS, 1)), Height * np.ones((NC, 1)), np.zeros((ND, 1)))).flatten()
    if NC > 8:
        UB2 = np.vstack((np.zeros((NS, 1)), Height * np.ones((4, 1)), np.zeros((NC-8, 1)), Height * np.ones((4, 1)), np.zeros((ND, 1)))).flatten()
    else:
        UB2 = np.vstack((np.zeros((NS, 1)), Height * np.ones((NC, 1)), np.zeros((ND, 1)))).flatten()

    UB = UB1 if UB_choice == 'UB1' else UB2

    # Hamiltonian matrix setup
    T = (2 * t0 * np.diag(np.ones((Np)))) - (t0 * np.diag(np.ones((Np-1)), 1)) - (t0 * np.diag(np.ones((Np-1)), -1))
    T = T + np.diagflat(UB)

    # Bias voltage setup
    NV = 26
    VV = np.linspace(0, V_opt, NV)  # Bias voltage range (V)

    # Initializing arrays
    TM = np.zeros(101)
    II = np.zeros(NV)

    for iV in range(NV):
        V = VV[iV]
        mu1 = Ef + (V / 2)
        mu2 = Ef - (V / 2)
        U1 = V * np.hstack((0.5 * np.ones(NS), np.linspace(0.5, -0.5, NC), -0.5 * np.ones(ND)))

        # Energy grid for Green’s function method
        NE = 101
        E = np.linspace(-0.2, 0.8, NE)
        zplus = 1j * 1e-12
        dE = E[1] - E[0]
        f1 = 1 / (1 + np.exp((E - mu1) / kT))
        f2 = 1 / (1 + np.exp((E - mu2) / kT))

        # Transmission
        I = 0  # Current

        for k in range(NE):
            sig1 = np.zeros((Np, Np), dtype=complex)
            sig2 = np.zeros((Np, Np), dtype=complex)
            sig3 = np.zeros((Np, Np), dtype=complex)

            ck = 1 - ((E[k] + zplus - U1[0] - UB[0]) / (2 * t0))
            ka = cmath.acos(ck)
            sig1[0, 0] = -t0 * np.exp(1j * ka)
            gam1 = 1j * (sig1 - conj(sig1.T))

            ck = 1 - ((E[k] + zplus - U1[Np-1] - UB[Np-1]) / (2 * t0))
            ka = cmath.acos(ck)
            sig2[Np-1, Np-1] = -t0 * np.exp(1j * ka)
            gam2 = 1j * (sig2 - conj(sig2.T))

            sig3[Np // 2 - 1, Np // 2 - 1] = -1j * 0.00025
            gam3 = 1j * (sig3 - conj(sig3.T))  # Büttiker probe

            G = inv(((E[k] + zplus) * eye(Np)) - T - diag(U1) - sig1 - sig2 - sig3)
            T12 = real(trace(dot(dot(dot(gam1, G), gam2), conj(G.T))))
            T13 = real(trace(dot(dot(dot(gam1, G), gam3), conj(G.T))))
            T23 = real(trace(dot(dot(dot(gam2, G), gam3), conj(G.T))))

            TM[k] = T12 + (T13 * T23 / (T12 + T23))
            I += dE * IE * TM[k] * (f1[k] - f2[k])

        II[iV] = I

    # Spatial positions in nanometers
    XX = a * 1e9 * np.array(list(range(1, Np + 1)))
    XS = XX[0:NS-4]
    XD = XX[NS+NC+5-1:Np]

    return VV, II, XX, XS, XD, U1, UB1, UB2, mu1, mu2

def plot_figures(VV, II, XX, XS, TM, E, XD, U1, UB1, UB2, mu1, mu2, V_opt, UB_choice):
    plt.figure(figsize=(14, 6))

    # Subplot 1: Potential profile
    plt.subplot(1, 3, 1)
    UB = UB1 if UB_choice == 'UB1' else UB2
    plt.plot(XX, U1 + UB, 'r', lw=1, label='Potential Profile')
    plt.plot(XS, mu1 * np.ones(len(XS)), 'r--', lw=1, label='$\mu_1$')
    plt.plot(XD, mu2 * np.ones(len(XD)), 'r--', lw=2, label='$\mu_2$')
    plt.xlabel(' $z$ ( nm ) $-->$ ', fontsize=15)
    plt.ylabel(' Energy ( eV ) $-->$ ', fontsize=15)
    # plt.xlim(0, 15)
    # plt.xlim(6.3, 9)
    plt.xlim(0, 15)
    plt.ylim(-0.4, 0.8)
    plt.tick_params(axis='both', which='major', labelsize=15, direction='in')
    # plt.xticks(np.arange(0, 20, 5))
    plt.grid(color='b', alpha=0.5, linestyle='--', lw=0.5)
    plt.legend()

    # Subplot 2: Transmission
    plt.subplot(1, 3, 2)
    plt.plot(TM, E, 'r', lw=2, label='Transmission')
    plt.xlabel(' Transmission $-->$ ', fontsize=15)
    plt.ylabel(' Energy ( eV ) $-->$ ', fontsize=15)
    plt.xlim(0, 1)
    plt.ylim(0, 0.8)
    plt.tick_params(axis='both', which='major', labelsize=15, direction='in')
    plt.xticks(np.arange(0.2, 1.2, 0.2))
    plt.grid(color='b', alpha=0.5, linestyle='--', lw=0.5)
    plt.legend()

    # Subplot 3: Current-Voltage Characteristics
    plt.subplot(1, 3, 3)
    plt.plot(VV, II, 'r', lw=2, label='I-V Curve')
    plt.xlabel(' Voltage ( V ) $-->$ ', fontsize=15)
    plt.ylabel(' Current ( A ) $-->$ ', fontsize=15)
    if V_opt == 0:
        plt.xlim(0, 0.1)  # If V_opt is 0, set x-axis limit to 0 to 0.1
    else:
        plt.xlim(0, V_opt)
    plt.ylim(0, 9e-7)
    plt.tick_params(axis='both', which='major', labelsize=15, direction='in')
    plt.xticks(np.arange(0, V_opt+0.1, 0.1) if V_opt != 0 else np.arange(0, 0.2, 0.1))
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    plt.grid(color='b', alpha=0.5, linestyle='--', lw=0.5)
    plt.legend()

    plt.show()

def interactive_plot(V_opt, NC, Height, UB_choice):
    XX, XS, XD, U1, UB1, UB2, TM, E, mu1, mu2 = transmission_IV(V_opt, NC, Height, UB_choice)
    VV, II, XX, XS, XD, U1,  UB1, UB2, mu1, mu2 = calculate_IV(V_opt, NC, Height, UB_choice)
    plot_figures(VV, II, XX, XS, TM, E, XD, U1, UB1, UB2, mu1, mu2, V_opt, UB_choice)

def update_NC(change):
    if UB_choice.value == 'UB1':
        NS_slider.value = ND_slider.value = (50 - NC_slider.value) // 2
    else:
        NS_slider.value = ND_slider.value = (50 - NC_slider.value) // 2

def update_UB_choice(change):
    if UB_choice.value == 'UB1':
        NC_slider.min = 4
        NC_slider.max = 20
        NC_slider.value = 4
    else:
        NC_slider.min = 12
        NC_slider.max = 40
        NC_slider.value = 16
    update_NC(None)


def update_label(change):
    a = change['new']
    width = 0.3 * (a)
    width_label.value = f'Width = {width:.2f} nm'


UB_choice = widgets.ToggleButtons(options=[('One barrier', 'UB1'), ('Two barrier', 'UB2')], description='Barrier Type')
NC_slider = widgets.IntSlider(min=4, max=15, step=1, value=3, description='NC')
width_label = widgets.Label(value=f'Width = {0.3 * (NC_slider.value):.2f} nm')

UB_choice.observe(update_UB_choice, names='value')
NC_slider.observe(update_NC, names='value')
NC_slider.observe(update_label, names='value')

Height_slider = widgets.FloatSlider(min=0.1, max=0.5, step=0.1, value=0.4, description='Height')
V_slider = widgets.FloatSlider(min=0.0, max=0.6, step=0.1, value=0.0, description='V')
NS_slider = widgets.IntText(value=(50 - NC_slider.value) // 2, description='NS', disabled=True)
ND_slider = widgets.IntText(value=(50 - NC_slider.value) // 2, description='ND', disabled=True)

ui = VBox([UB_choice, Height_slider, HBox([NC_slider, width_label]), V_slider])
out = interactive_output(interactive_plot, {'UB_choice': UB_choice, 'V_opt': V_slider, 'NC': NC_slider, 'Height': Height_slider})

display(ui, out)



Output()